# Generating Features



Let's recall: We want to learn how to fill in a missing residue, or more precisely, fill in the coordinates of a missing residue. Ultimately, we want to rebuild an entire amino acid, but let's start with just the $C\alpha$ coordinates. We can later extend the machine learning method to also rebuild $C\beta$ and other coordinates and/or use other tools to complete missing coordinates from $C\alpha$ and $C\beta$ coordinates. 

So the first problem we will tackle is as follows:

Given the $C\alpha$ coordinates of a chain of N amino acids, can we predict the coordinates of one of the $C\alpha$ coordinates based on the coordinates of the other ones if it is removed?

We will make these even simpler: Can we predict the coordinates of one of the $C\alpha$ coordinates based on the coordinates of the *two* $C\alpha$ atoms before and after in the chain?

So let's say, we remove $C\alpha_i$, can we learn how to predict its coordinates from the coordinates of $C\alpha_{i-2}$, $C\alpha_{i-1}$, $C\alpha_{i+1}$, $C\alpha_{i+2}$?

It is best to describe the problem using **local** coordinates instead of the XYZ coordinates from the PDB file. The reason is that the XYZ coordinates in the PDB are subject to molecular translation or rotation and it makes the problem unnecessarily difficult for the machine learning if we use such coordinates.

We will construct lcoal coordinates as follows:

1) We define a vector $l_1$ between $C\alpha$ coordinates $ca_{i-1}$ and $ca_i+1$ 

$$
l_1 = ca_{i+1} - ca_{i-1}
$$

In this notation, both $l$ and $ca$ are vectors with three values (x, y, z). So the above equation is short-hand for:

$$
\left(
\begin{matrix}
l_{1,x} \\
l_{1,y} \\
l_{1,z} \\
\end{matrix}
\right)
=
\left(
\begin{matrix}
ca_{i+1,x} \\
ca_{i+1,y} \\
ca_{i+1,z} \\
\end{matrix}
\right)
-
\left(
\begin{matrix}
ca_{i-1,x} \\
ca_{i-1,y} \\
ca_{i-1,z} \\
\end{matrix}
\right)
$$

2) The midpoint $m$ of $l_1$ is given by:

$$
m = ca_{i-1}+\frac{1}{2}l_1
$$

or 

$$
m = ca_{i-1}+\frac{1}{2}(ca_{i+1}-ca_{i-1}) = \frac{1}{2}(ca_{i+1}+ca_{i-1})
$$

We use the **midpoint** $m$ as the origin of our local coordinate system.

3) We define an additional vector $l_2$ based on $ca_{i-2}$ and $ca_{i+2}$:

$$
l_2 = (ca_{i-2}-ca_{i-1}) + (ca_{i+2}-ca_{i+1})
$$

4) To obtain a third vector $l_3$ that is orthogonal to $l_1$ and $l_2$ we apply the [cross product](https://en.wikipedia.org/wiki/Cross_product):

$$
l_3 = l_1 x l_2
$$

5) We apply the cross product again to obtain an orthogonal vector $ol_2$ from $l_3$ and $l_1$:

$$
ol_2 = l_3 x l_1
$$

6) After normalization (meaning the length of each vector is 1), this gives us an orthogonal local coordinate system (meaning all vectors are at right angles to each other), defined by the $C\alpha$ coordinates of residues i-2, i-1, i+1, and i+2:  

$$
x_{local} = \frac{l_1}{|l_1|}
y_{local} = \frac{ol_2}{|ol_2|}
z_{local} = \frac{l_3}{|l_3|}
$$

$|l_1|$ is the norm of a vector calculated as:

$$
|l_1| = \sqrt(l_{1,x}^2 + l_{1,y}^2 + l_{1,z}^2)
$$

So what we will do next is to go through our list of PDBs and then through each residue and pretend that we remove that residue. We then calculate $m$, $x_{local}$, $y_{local}$, and $z_{local}$ as described above followed by projection of the coordinates of the 'removed' $C\alpha$ - the target that we want to learn how to predict - as well as projections of other $C\alpha$s - and later other atoms - which will be our input features for the machine learning.    


In [ ]:
import os
import mdtraj as md
import numpy as np

tag='longer'

# projection function used below
def project(vector,midpoint,xloc,yloc,zloc):
    vmid=vector-midpoint
    return [np.dot(vmid,xloc), np.dot(vmid,yloc), np.dot(vmid,zloc)]

pdblist_filename = f"../extractingpdbs/{tag}_clean_pdb_chain.txt"
pdblist=open(pdblist_filename)

chain_directory = "../extractingpdbs/chains"

# file name to write output to
target_feature_filename = f"{tag}_local_cai_aa_capm2.dat"
target_feature = open(target_feature_filename,"w")

for line in pdblist:
    items=line.split()
    pdb=items[0]
    chain=items[1]
    
    chain_filename=f"{chain_directory}/{pdb}_{chain}.pdb"
        
    # check whether we can find the prepared PDB file
    if (not os.path.isfile(chain_filename)):
        print(f"cannot find {chain_filename}, skipping")
    else:
        # loading PDB file using mdtraj
        pdb=md.load_pdb(chain_filename)

        # extracting coordinates for C-alpha coordinates
        caxyz=[]
        calist=pdb.topology.select("name CA")
        for ca in calist:
            caxyz=np.append(caxyz,pdb.xyz[0][ca])
        caxyz=np.reshape(caxyz,(-1,3))

        # go through all residues (excluding the first two and last two residues):
        for i in range(2,len(caxyz)-2):
            ca_i=caxyz[i]
            ca_m1=caxyz[i-1]
            ca_m2=caxyz[i-2]
            ca_p1=caxyz[i+1]
            ca_p2=caxyz[i+2]

            # midpoint, see above
            m=0.5*(ca_m1+ca_p1)
            
            # local coordinate system, see above
            l1=ca_p1-ca_m1
            l2=(ca_m2-ca_m1)+(ca_p2-ca_p1)
            l3=np.cross(l1,l2)
            ol2=np.cross(l3,l1)

            xlocal=l1/np.linalg.norm(l1)
            ylocal=ol2/np.linalg.norm(ol2)
            zlocal=l3/np.linalg.norm(l3)
            
            # projection of Ca_i onto local coordinates
            # this is the TARGET that we want to predict
            ca_i_local=project(ca_i,m,xlocal,ylocal,zlocal)
            # could add Cbeta or other atoms later
            
            if (ca_i_local[0]<-1 or ca_i_local[0]>1 or ca_i_local[1]<-1 or ca_i_local[1]>1 or ca_i_local[2]<-1 or ca_i_local[1]>1):
                print(f"coordinates out of range {chain_filename} residue {i}")
                print(*ca_i_local)
                print("\n")
            else:
                # projection of other Ca atoms onto local coordinates
                # these are the INPUT FEATURES that we use to make the prediction
                ca_m1_local=project(ca_m1,m,xlocal,ylocal,zlocal)
                ca_m2_local=project(ca_m2,m,xlocal,ylocal,zlocal)
                ca_p1_local=project(ca_p1,m,xlocal,ylocal,zlocal)
                ca_p2_local=project(ca_p2,m,xlocal,ylocal,zlocal)
                # could be others such as Cbetas, or closest 10 atoms in space
                input_feature=ca_m1_local+ca_m2_local+ca_p1_local+ca_p2_local
            
                # another optional feature could be the amino acid type of Ca_i
                aa=pdb.topology.atom(calist[i]).residue.name
            
                print(*ca_i_local, aa, *input_feature, file=target_feature)
            
        print(f"worked on {chain_filename}")
        
target_feature.close()
        